<a href="https://colab.research.google.com/github/vaasu14/pyspark/blob/main/Tree_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-3.1.2//spark-3.1.2-bin-hadoop3.2.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz



In [2]:
cd /content/spark-3.1.2-bin-hadoop3.2

/content/spark-3.1.2-bin-hadoop3.2


In [3]:

!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"
import findspark
findspark.init()
findspark.find()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('pyspark').getOrCreate()


In [4]:
cd /content/spark-3.1.2-bin-hadoop3.2

/content/spark-3.1.2-bin-hadoop3.2


In [5]:
from google.colab import files
upload=files.upload()

Saving tic-tac-toe-endgame.csv to tic-tac-toe-endgame.csv


In [6]:
df=spark.read.csv('/content/spark-3.1.2-bin-hadoop3.2/tic-tac-toe-endgame.csv',inferSchema=True,header=True)

This database encodes the complete set of possible board configurations at the end of tic-tac-toe games, where "x" is assumed to have played first. The target concept is "win for x" (i.e., true when "x" has one of 8 possible ways to create a "three-in-a-row").

Attribute Information

V1 = top-left-square: {x,o,b}

V2 = top-middle-square: {x,o,b}

V3 = top-right-square: {x,o,b}

V4 = middle-left-square: {x,o,b}

V5 = middle-middle-square: {x,o,b}

V6 = middle-right-square: {x,o,b}

V7 = bottom-left-square: {x,o,b}

V8 = bottom-middle-square: {x,o,b}

V9 = bottom-right-square: {x,o,b}

V10 = Class: {positive,negative}

In [7]:
df.head(1)

[Row(V1='x', V2='x', V3='x', V4='x', V5='o', V6='o', V7='x', V8='o', V9='o', V10='positive')]

In [8]:
df.printSchema()

root
 |-- V1: string (nullable = true)
 |-- V2: string (nullable = true)
 |-- V3: string (nullable = true)
 |-- V4: string (nullable = true)
 |-- V5: string (nullable = true)
 |-- V6: string (nullable = true)
 |-- V7: string (nullable = true)
 |-- V8: string (nullable = true)
 |-- V9: string (nullable = true)
 |-- V10: string (nullable = true)



In [9]:
df.head(1)

[Row(V1='x', V2='x', V3='x', V4='x', V5='o', V6='o', V7='x', V8='o', V9='o', V10='positive')]

In [10]:
from pyspark.ml.feature import (VectorAssembler,StringIndexer,OneHotEncoder)

In [11]:
v_SI=StringIndexer(inputCols=['V1','V2','V3','V4','V5','V6','V7','V8','V9'],outputCols=['V1_SI','V2_SI','V3_SI','V4_SI','V5_SI','V6_SI','V7_SI','V8_SI','V9_SI'])

In [12]:
pred_SI=StringIndexer(inputCol='V10',outputCol='V10_SI')

In [13]:
v_OE=OneHotEncoder(inputCols=['V1_SI','V2_SI','V3_SI',
                             'V4_SI','V5_SI','V6_SI','V7_SI','V8_SI','V9_SI'],outputCols=['V1_E','V2_E','V3_E','V4_E','V5_E','V6_E','V7_E','V8_E','V9_E'])

In [14]:
assembler=VectorAssembler(inputCols=['V1_E','V2_E','V3_E','V4_E','V5_E','V6_E','V7_E','V8_E','V9_E'],outputCol='features')

In [15]:
df.printSchema()

root
 |-- V1: string (nullable = true)
 |-- V2: string (nullable = true)
 |-- V3: string (nullable = true)
 |-- V4: string (nullable = true)
 |-- V5: string (nullable = true)
 |-- V6: string (nullable = true)
 |-- V7: string (nullable = true)
 |-- V8: string (nullable = true)
 |-- V9: string (nullable = true)
 |-- V10: string (nullable = true)



In [16]:
from pyspark.ml.classification import (DecisionTreeClassifier,RandomForestClassifier,GBTClassificationModel)

In [17]:
from pyspark.ml import Pipeline

In [18]:
rfc=RandomForestClassifier(featuresCol='features',labelCol='V10_SI')

In [19]:
pipeline=Pipeline(stages=[v_SI,pred_SI,v_OE,assembler,rfc])

In [24]:
train_data,test_data=df.randomSplit([0.7,0.3])

In [25]:
rfc_model=pipeline.fit(train_data)

In [26]:
results=rfc_model.transform(test_data)

In [27]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [28]:
binary_eval=BinaryClassificationEvaluator(labelCol='V10_SI')

In [29]:
print(binary_eval.evaluate(results))

0.9526300494042428
